In [97]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# two hyperparameters
block_size = 8
batch_size = 4

cpu


In [6]:
with open ('WizardOfOz.txt' , 'r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [7]:
string_to_int = {}
int_to_string = {}
index = 0;
#generating mappings 
for ch in chars:
    string_to_int[ch] = index
    int_to_string[index] = ch
    index += 1
#encoding of given string into integers
def encode(s):
    result = []
    for c in s:
        result.append(string_to_int[c])
    return result
#decoding of the given integer array to the original string 
def decode(l):
    result = ''
    for i in l:
        result += int_to_string[i]
    return result



# this is a character level tokeniser and this converts each character to integer equivalent 


In [9]:
#in character level tokeniser we have huge amount of tokens to encode or decode 
#but a very small amount of vacabulary {a , b , c ,......}

#in word level tokeniser we have a lot of words hence a large amount of vacab and small amount of  


In [8]:
data = torch.tensor(encode(text) , dtype = torch.long)  # tensors are a data structures that pytorch uses 
print(data)

tensor([80,  0,  0,  ..., 67, 57,  0])


In [96]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs')
#print(x.shape)
print(x)
print('targets:')
print(y)


tensor([ 18833,  33470, 120777, 174688])
inputs
tensor([[73, 61, 62, 67, 60,  1, 67, 68],
        [61, 58, 71, 58,  1, 55, 58,  1],
        [61, 58, 54, 57,  9,  1, 34, 62],
        [72,  9,  1, 31, 62, 65, 65, 62]])
targets:
tensor([[61, 62, 67, 60,  1, 67, 68, 76],
        [58, 71, 58,  1, 55, 58,  1, 54],
        [58, 54, 57,  9,  1, 34, 62, 66],
        [ 9,  1, 31, 62, 65, 65, 62, 64]])


In [12]:
block_size = 8;

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ',context,'target is', target)


when input is  tensor([80]) target is tensor(0)
when input is  tensor([80,  0]) target is tensor(0)
when input is  tensor([80,  0,  0]) target is tensor(1)
when input is  tensor([80,  0,  0,  1]) target is tensor(1)
when input is  tensor([80,  0,  0,  1,  1]) target is tensor(28)
when input is  tensor([80,  0,  0,  1,  1, 28]) target is tensor(39)
when input is  tensor([80,  0,  0,  1,  1, 28, 39]) target is tensor(42)
when input is  tensor([80,  0,  0,  1,  1, 28, 39, 42]) target is tensor(39)


In [100]:
class BigramLanguageModel(nn.Module):# basically we are making bigram language model class as a subclass of nn.module which has all the important aspect of a neural network including layers , activation functions etc 
     def __init__(self,vocab_size):
         super().__init__()
         self.token_embedding_table = nn.Embedding(vocab_size,vocab_size) # so basically its like a look up table for each words index we have randomly initialised a vector and each vector is represented by the size of 4 

     def forward(self,index,targets):
         logits = self.token_embedding_table(index)
         #logits are normalised tensor with probabilities 
         #say a tensor is [0.16 , 0.33, 0.5] ans say these are linked to 3 bigrams say 'ab' , 'ac' , 'ad'  it means the chances of 'ad' is high 
         #probability distribution of what comes next is logits

         B,T,C = logits.shape
         logits = logits.view(B*T , C)
         targets = targets.view(B*T)
         loss = F.cross_entropy(logits,targets)
         return logits

In [ ]:
#Gradient Descent is an optimiser it will optimise our parameters wirgth matrices etc 